In [ ]:
import itertools, requests, re, scipy.optimize
from bs4 import BeautifulSoup
 
BEN_SLOTS = set(itertools.chain(
                range(36,52),                                                   #Ben - Sunday
                range(132,192),                                                 #Ben - Monday
                range(228,242),range(251,288),                                  #Ben - Tuesday
                range(324,338),range(366,384),                                  #Ben - Wednesday
                range(420,434),range(462,480),                                  #Ben - Thursday
                range(516,576),                                                 #Ben - Friday
                range(612,672)                                                  #Ben - Saturday
                ))
MICHAEL_SLOTS = set(itertools.chain(
                list(),                                                         #Michael - Sunday
                range(144,192),                                                 #Michael - Monday
                range(240,264),                                                 #Michael - Tuesday
                range(340,384),                                                 #Michael - Wednesday
                range(432,456),                                                 #Michael - Thursday
                range(528,544),                                                 #Michael - Friday
                list()                                                          #Michael - Saturday
                ))
MOYI_SLOTS = set(itertools.chain(
                list(),                                                         #Moyi - Sunday
                range(132,144),range(172,184),                                  #Moyi - Monday
                range(228,240),range(264,280),                                  #Moyi - Tuesday
                range(324,336),range(360,376),                                  #Moyi - Wednesday
                range(420,432),range(456,472),                                  #Moyi - Thursday
                range(516,528),range(552,568),                                  #Moyi - Friday
                list()                                                          #Moyi - Saturday
                ))
TERESSA_SLOTS = set(itertools.chain(
                list(),                                                         #Teressa - Sunday
                range(140,160),                                                 #Teressa - Monday
                list(),                                                         #Teressa - Tuesday
                range(332,352),                                                 #Teressa - Wednesday
                list(),                                                         #Teressa - Thursday
                range(524,544),                                                 #Teressa - Friday
                list()                                                          #Teressa - Saturday
                ))
CLASS_TIME_SLOTS = set(itertools.chain(
                range(228,234),                                                 #Tuesday class
                range(420,426)                                                  #Thursday class
                ))
CARLY_HOURS = set(range(244,248))                                               #Carly's hours are Tuesday, 13:00-14:00
 
 
def download_page(pageid):
    resp = requests.get(f'https://www.when2meet.com/?{pageid}')
    soup = BeautifulSoup(resp.content, 'lxml')
    soup_str = str(soup)
    student_id_defs = re.findall('PeopleIDs\[[0-9]+\] = [0-9]+;', soup_str)
    student_name_defs = re.findall('PeopleNames\[[0-9]+\] = .*?;', soup_str)
    students_temp = dict()
    for id_def in student_id_defs:
        temp_num = int(id_def.split('[')[1].split(']')[0])
        student_id = int(id_def.split('=')[1][1:-1])
        students_temp[temp_num] = {'id': student_id}
    for id_def in student_name_defs:
        temp_num = int(id_def.split('[')[1].split(']')[0])
        name = id_def.split('=')[1][2:-2]
        students_temp[temp_num]['name'] = name
    students = {temp['id']: temp['name'] for temp_num, temp in students_temp.items()}
    slot_defs = re.findall('TimeOfSlot\[[0-9]+\]=[0-9]+;', soup_str)
    pushes = re.findall('AvailableAtSlot\[[0-9]+\]\.push\([0-9]+\);', soup_str)
    slots = dict()
    for i in {int(slot_def.split('[')[1].split(']')[0]) for slot_def in slot_defs}:
        slots[i] = set()
    for push in pushes:
        slot = int(push.split('[')[1].split(']')[0])
        student_id = int(push.split('(')[1].split(')')[0])
        slots[slot].add(student_id)
    for student in list(students):
        if not [slot for slot, slot_students in slots.items() if student in slot_students]:
            del students[student]
    return students, slots
 
def interpret_slot(slot):
    day = slot // 96
    time = slot % 96
    days = [
            'Sunday',
            'Monday',
            'Tuesday',
            'Wednesday',
            'Thursday',
            'Friday',
            'Saturday'
            ]
    hour = time // 4
    minute = 15*(time % 4)
    return f'{days[day]} at {hour}:{str(minute).zfill(2)}'
 
def invert_slots(students, slots):                                              #Because I asked students to mark when they're unavailable.
    new_slots = dict()
    student_ids = set(students.keys())
    for slot in slots.keys():
        new_slots[slot] = student_ids.difference(slots[slot])
    return new_slots
 
def points_eval(x):
    c = x // 4                                                                  #Number of complete hours at full valuation that a student can attend.
    e = x % 4                                                                   #Number of 15-minute slots at full valuation beyond c hours that a student can attend.
    return 16/3-1/3*4**(2-c)*(4**c-1)-e/4**c                                    #Four 15-minute slots in the second hour are weighted as much as one slot in the first hour, etc.
 
def evaluate_hours(hours_slots, students, student_slots):
    points = {student: 0 for student in students}
    for slot, slot_students in student_slots.items():
        if ([hours for hours in hours_slots if slot in hours] and slot not in CLASS_TIME_SLOTS) \
        or slot in CARLY_HOURS:
            if 426 <= slot < 576:
                point_val = 0.5                                                 #Office hours on Thursday and Friday (i.e. right after assignments are due) are worth much less.
            elif slot >= 576:
                point_val = 0.75                                                #Office hours on Saturday are worth a little less.
            else:
                point_val = 1
            for student in slot_students:
                points[student] += point_val
    return sum(points_eval(x) for student, x in points.items()), points
 
def objective(staff, students, student_slots):
    ben = int(staff[0])
    michael = int(staff[1])
    moyi = int(staff[2])
    teressa = int(staff[3])
    hours_slots = [
                   {i % 672 for i in range(ben,ben+8) if i in BEN_SLOTS},             #Assuming everyone gives two full consecutive hours.
                   {i % 672 for i in range(michael,michael+8) if i in MICHAEL_SLOTS},
                   {i % 672 for i in range(moyi,moyi+8) if i in MOYI_SLOTS},
                   {i % 672 for i in range(teressa,teressa+8) if i in TERESSA_SLOTS},
                  ]
    return evaluate_hours(hours_slots, students, student_slots)[0]
 
def optimize_hours(students, student_slots):
    results = scipy.optimize.dual_annealing(objective, [(0,672)]*4,
                                            args=(students, student_slots),
                                            maxiter=5000,
                                            initial_temp=50000)
    x = results['x']
    result_slots = [interpret_slot(int(i)) for i in x]
    print(f'Value of objective function is {results["fun"]}')
    print(f'Ben works {result_slots[0]}')
    print(f'Michael works {result_slots[1]}')
    print(f'Moyi works {result_slots[2]}')
    print(f'Teressa works {result_slots[3]}')
    return results

In [ ]:
students, slots = download_page('9735283-FkhoO')

In [ ]:
slots = invert_slots(students, slots)

In [ ]:
students_who_filled_out_the_form_in_reverse = [42402232,42449648,42420741,42607415]

In [ ]:
for slot in slots.keys():
    for student in students_who_filled_out_the_form_in_reverse:
        if student in slots[slot]:
            slots[slot] = slots[slot].difference({student})
        else:
            slots[slot].add(student)

In [ ]:
results = optimize_hours(students, slots)

Value of objective function is 1.1472066243489403
Ben works Sunday at 10:00
Michael works Wednesday at 16:00
Moyi works Tuesday at 19:30
Teressa works Monday at 11:00


In [ ]:
staff = results['x']

In [ ]:
ben = int(staff[0])
michael = int(staff[1])
moyi = int(staff[2])
teressa = int(staff[3])
hours_slots = [
               {i for i in range(ben,ben+8) if i in BEN_SLOTS},                 #Assuming everyone gives two full consecutive hours.
               {i for i in range(michael,michael+8) if i in MICHAEL_SLOTS},
               {i for i in range(moyi,moyi+8) if i in MOYI_SLOTS},
               {i for i in range(teressa,teressa+8) if i in TERESSA_SLOTS},
              ]

In [ ]:
test, test_points = evaluate_hours(hours_slots, students, slots)

In [ ]:
test_points

{42400247: 34,
 42400574: 20,
 42400647: 20,
 42400805: 20,
 42400866: 32,
 42401075: 28,
 42401946: 31,
 42402232: 24,
 42404193: 32,
 42405286: 32,
 42405369: 26,
 42405673: 18,
 42405728: 32,
 42405775: 28,
 42406224: 30,
 42412318: 32,
 42412390: 26,
 42418010: 20,
 42418990: 36,
 42419241: 28,
 42420741: 10,
 42420965: 20,
 42425050: 28,
 42428223: 32,
 42428239: 28,
 42429302: 20,
 42431134: 30,
 42435718: 8,
 42437804: 22,
 42438596: 18,
 42439236: 26,
 42442004: 28,
 42449369: 12,
 42449648: 10,
 42450134: 18,
 42453032: 34,
 42469805: 24,
 42473708: 24,
 42477910: 22,
 42488634: 16,
 42497985: 28,
 42499681: 32,
 42500330: 32,
 42500331: 32,
 42500344: 34,
 42500352: 20,
 42500846: 28,
 42500959: 28,
 42502078: 32,
 42502471: 22,
 42503089: 14,
 42508171: 32,
 42527201: 20,
 42551553: 24,
 42594987: 20,
 42600980: 24,
 42603551: 24,
 42607415: 36,
 42609992: 6,
 42615203: 32,
 42617084: 20}

In [ ]:
from statistics import mean; [min(test_points.values()), mean(test_points.values())]

[8, 25.0327868852459]